In [168]:
import json
import os
import statistics

import numpy as np
import pandas as pd
from tqdm import tqdm

tqdm.pandas()
pd.options.display.max_columns = None

In [15]:
scenarios_dir = '../../data/processed/scenarios/'
scenario_dir = 'scenarios_1_1_pub_trans_comfort_dist-up_0_0_pub_trans_punctuality_dist-up_0_0_household_cars_dist-down_0_0/'

scenario_path = os.path.join(scenarios_dir, scenario_dir)

In [73]:
agents_examp = pd.read_csv(os.path.join(scenario_path, 'agents_results_1.csv'))

agents_examp.head()

,Step,AgentID,agent_id,home_region,age_sex,pub_trans_comfort,pub_trans_punctuality,bicycle_infrastr_comfort,pedestrian_inconvenience,household_persons,household_cars,household_bicycles,travels_num
0,0,0,0,228,25-44_K,1.0,2.0,1.0,0.0,5.0,1.0,4.0,2
1,0,1,1,19,25-44_K,3.0,2.0,4.0,2.0,1.0,1.0,1.0,2
2,0,2,2,116,61-x_K,2.0,3.0,2.0,2.0,2.0,0.0,0.0,2
3,0,3,3,1,45-60_K,2.0,1.0,3.0,2.0,3.0,1.0,2.0,0
4,0,4,4,55,25-44_K,2.0,0.0,3.0,0.0,2.0,1.0,0.0,2


In [74]:
travels_examp = pd.read_csv(os.path.join(scenario_path, 'travels_results_1.csv'), index_col=0)

travels_examp.head()

,agent_id,start_region,start_place_type,dest_region,dest_place_type,travel_start_time,transport_mode,is_driver
0,0,228,dom,126,praca,1219,0,1.0
1,0,126,praca,228,dom,1229,0,1.0
2,1,19,dom,1,inne,977,1,NaN
3,1,1,inne,19,dom,987,1,NaN
4,2,116,dom,147,inne,397,2,NaN


In [ ]:
{
    ‘grupa_wiek_plec’: {
		‘modal_split’: {
			‘komunikacja’: {
                średnia: średnia liczba przejazdów komunikacją,
                ‘std’: wartość
            }
			‘auta_kierowca’: jw
			‘auta_nie_kierowca’: jw
			itd.
		},
		‘rozklad na cele podrozy’: {
			‘dom’: {
                ‘średnia’: średnia liczba podróży do domu
                ‘std’: std:
            }
			itd.
		},
		‘liczba podróży’: {
            średnia
        },
        pub_trans_comfort: {
            średnia ocena
        },
        pub_trans_punctuality: {
            średnia ocena
        },
        bicycle_infrastr_comfort: {
            średnia ocena
        },
        pedestrian_inconvenience: {
            średnia
        },
        household_persons: {
            średnia
        },
        household_cars: {
            średnia
        },
        household_bicycles: {
            średnia
        }
	}
}

## Scenario 0 0 0 

In [140]:
scenarios_dir = '../../data/processed/scenarios/'
scenario_dir = 'scenarios_1_1_pub_trans_comfort_dist-up_0_0_pub_trans_punctuality_dist-up_0_0_household_cars_dist-down_0_0/'

scenario_path = os.path.join(scenarios_dir, scenario_dir)

In [141]:
simulations_files = [f for f in os.listdir(scenario_path) if f.startswith(('agents_results_', 'travels_results_'))]
num_simulations = int(len(simulations_files) / 2)

num_simulations

50

In [142]:
unique_age_sex = [
    '6-15_K',
    '6-15_M',
    '16-19_K',
    '16-19_M',
    '20-24_K',
    '20-24_M',
    '25-44_K',
    '25-44_M',
    '45-60_K',
    '45-65_M',
    '61-x_K',
    '66-x_M'
]

unique_transport_mode = [
    'car_driver',
    'car_passenger',
    'public',
    'pedestrian',
    'bicycle'
]

unique_dest_place_type = [
    'dom', 
    'inne', 
    'praca', 
    'szkola', 
    'uczelnia'
]

other_params = [
    'travels_num',
    'pub_trans_comfort',
    'pub_trans_punctuality',
    'bicycle_infrastr_comfort',
    'pedestrian_inconvenience',
    'household_persons',
    'household_cars',
    'household_bicycles'
]

statistics = [
    'mean', 
    'std'
]


json_dict = {}

for age_sex_comb in unique_age_sex:
    json_dict[age_sex_comb] = {}

    json_dict[age_sex_comb]['modal_split'] = {}
    for tm in unique_transport_mode:
        json_dict[age_sex_comb]['modal_split'][tm] = {}
        for stat in statistics:
            json_dict[age_sex_comb]['modal_split'][tm][stat] = []
    
    json_dict[age_sex_comb]['dest_place_type'] = {}
    for dpt in unique_dest_place_type:
        json_dict[age_sex_comb]['dest_place_type'][dpt] = {}
        for stat in statistics:
            json_dict[age_sex_comb]['dest_place_type'][dpt][stat] = []
    
    for param in other_params:
        json_dict[age_sex_comb][param] = {}
        for stat in statistics:
            json_dict[age_sex_comb][param][stat] = []

In [143]:
def transport_mode_count(df_travels, age_sex_comb, tm):
    if tm.startswith('car_'):
        if tm == 'car_driver':
            transport_mode = 0
            is_driver = 1.0
        else:
            transport_mode = 0
            is_driver = 0.0

        count = df_travels[
            (df_travels['age_sex'] == age_sex_comb) &
            (df_travels['transport_mode'] == transport_mode) &
            (df_travels['is_driver'] == is_driver)
        ].shape[0]
    else:
        if tm == 'public':
            transport_mode = 1
        elif tm == 'pedestrian':
            transport_mode = 2
        elif tm == 'bicycle':
            transport_mode = 3
        else:
            transport_mode = -1
        
        count = df_travels[
            (df_travels['age_sex'] == age_sex_comb) &
            (df_travels['transport_mode'] == transport_mode)
        ].shape[0]
    
    return count


def dest_place_type_count(df_travels, age_sex_comb, dpt):
    count = df_travels[
        (df_travels['age_sex'] == age_sex_comb) &
        (df_travels['dest_place_type'] == dpt)
    ].shape[0]
    
    return count


def get_travels_num(df_travels, age_sex_comb):
    count = df_travels[
        df_travels['age_sex'] == age_sex_comb
    ].shape[0]
    
    return count


def get_avg_param(df_agents, age_sex_comb, param):
    df_agents_filtred = df_agents[
        df_travels['age_sex'] == age_sex_comb
    ]
    avg = df_agents_filtred[param].mean()

    return avg

In [144]:
simulations_files = [f for f in os.listdir(scenario_path) if f.startswith(('agents_results_', 'travels_results_'))]
num_simulations = int(len(simulations_files) / 2)

# for simulation in tqdm(range(1, 3)):
for simulation in tqdm(range(1, num_simulations+1)):
    agents_file = 'agents_results_' + str(simulation) + '.csv'
    travels_file = 'travels_results_' + str(simulation) + '.csv'
    df_agents = pd.read_csv(os.path.join(scenario_path, agents_file))
    df_travels = pd.read_csv(os.path.join(scenario_path, travels_file), index_col=0)

    df_travels = pd.merge(df_travels, df_agents[['agent_id','age_sex']], on='agent_id', how='left') # left - means agents who have not attempted to travel do not exist in this df

    for age_sex_comb in unique_age_sex:
        for tm in unique_transport_mode:
            count = transport_mode_count(df_travels, age_sex_comb, tm)
            for stat in statistics:
                json_dict[age_sex_comb]['modal_split'][tm][stat].append(count)
        
        for dpt in unique_dest_place_type:
            count = dest_place_type_count(df_travels, age_sex_comb, dpt)
            for stat in statistics:
                json_dict[age_sex_comb]['dest_place_type'][dpt][stat].append(count)
        
        for param in other_params:
            if param == 'travels_num':
                count = get_travels_num(df_travels, age_sex_comb)
                for stat in statistics:
                    json_dict[age_sex_comb][param][stat].append(count)
            else:
                avg = get_avg_param(df_agents, age_sex_comb, param)
                for stat in statistics:
                    json_dict[age_sex_comb][param][stat].append(avg)

100%|██████████| 50/50 [10:11<00:00, 12.24s/it]


In [164]:
for age_sex_comb in tqdm(unique_age_sex):
    for tm in unique_transport_mode:
        json_dict[age_sex_comb]['modal_split'][tm]['mean'] = statistics.mean(json_dict[age_sex_comb]['modal_split'][tm]['mean'])
        json_dict[age_sex_comb]['modal_split'][tm]['std'] = statistics.stdev(json_dict[age_sex_comb]['modal_split'][tm]['std'])
    
    for dpt in unique_dest_place_type:
        json_dict[age_sex_comb]['dest_place_type'][dpt]['mean'] = statistics.mean(json_dict[age_sex_comb]['dest_place_type'][dpt]['mean'])
        json_dict[age_sex_comb]['dest_place_type'][dpt]['std'] = statistics.stdev(json_dict[age_sex_comb]['dest_place_type'][dpt]['std'])
    
    for param in other_params:
        json_dict[age_sex_comb][param]['mean'] = statistics.mean(json_dict[age_sex_comb][param]['mean'])
        json_dict[age_sex_comb][param]['std'] = statistics.stdev(json_dict[age_sex_comb][param]['std'])

100%|██████████| 12/12 [00:00<00:00, 160.35it/s]


In [165]:
json_dict['6-15_K']

{'modal_split': {'car_driver': {'mean': 0, 'std': 0.0},
  'car_passenger': {'mean': 9751.28, 'std': 139.34233867327896},
  'public': {'mean': 6932.94, 'std': 105.70471192169713},
  'pedestrian': {'mean': 12078.8, 'std': 131.554287699964},
  'bicycle': {'mean': 2737.1, 'std': 80.35844443390336}},
 'dest_place_type': {'dom': {'mean': 15995.16, 'std': 111.52987336611311},
  'inne': {'mean': 2349.56, 'std': 45.36750612767797},
  'praca': {'mean': 62.22, 'std': 8.313475660096708},
  'szkola': {'mean': 13029.9, 'std': 100.35371117671123},
  'uczelnia': {'mean': 63.28, 'std': 9.833387524352004}},
 'travels_num': {'mean': 31500.12, 'std': 218.6985288716233},
 'pub_trans_comfort': {'mean': 2.6356936096937518,
  'std': 0.005949268432362485},
 'pub_trans_punctuality': {'mean': 2.5312508928600477,
  'std': 0.0063625681958572565},
 'bicycle_infrastr_comfort': {'mean': 2.6565470517229017,
  'std': 0.005698577130883263},
 'pedestrian_inconvenience': {'mean': 1.5314335902857876,
  'std': 0.00724594477

In [166]:
out_dir = '../../data/processed/mc/scenarios'

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [169]:
out_file = 'scenario_0_0_0.json'
out_path = os.path.join(out_dir, out_file)

with open(out_path, 'w') as f:
    json.dump(json_dict, f)